# Backend Demo pytreedb

## Import data

Package imports

In [ ]:
import pytreedb.db as pytreedb
import json
import shapely
import folium

# import matplotlib.pyplot as plt

Initiate a PyTreeDB and import data from ZIP web resource

In [ ]:
db = pytreedb.PyTreeDB(dbfile=r"tmp\SYSSIFOSS\test.db")
db.import_data(
    r"https://heibox.uni-heidelberg.de/f/05969694cbed4c41bcb8/?dl=1", overwrite=True
)

## Print statistics

Print statistics (number of trees and species)

In [ ]:
print(db.get_stats())

Print a list of species.

In [ ]:
print(db.get_list_species())

## Query the DB

- Query by species

In [ ]:
query_res = db.query_by_species("Quercus")
print("Number of trees of Genus Quercus:", len(query_res))

In [ ]:
print("One example:\n", json.dumps(query_res[0], indent=4))

In [ ]:
query_res = db.query_by_species("Abies alba")
print(db.get_ids(query_res))

- Query by canopy condition

In [ ]:
query_res = db.query("canopy_condition", "leaf-off", False)  # exact value search
print("Number of trees with data in condition 'leaf-off' :", len(query_res))

- Query by quality indicator: Get all trees with point clouds of quality 2 or higher.

In [ ]:
print(
    "Query by numeric comparison - quality <= 2: ",
    len(db.query_by_numeric_comparison("quality", 2, pytreedb.le)),
)

- Query by existence of a tree metric (e.g. DBH)

In [ ]:
print("Number of trees with property 'DBH_cm': ", len(db.query_by_key_exists("DBH_cm")))

- Query by acquisition date

In [ ]:
print(
    "Number of trees acquired on or after 2020-04-01: ",
    len(db.query_by_date("date", "2020-04-01", pytreedb.ge)),
)

- Query by geometry

In [ ]:
Bretten_coords = [
    [
        [8.679647386702298, 49.019748234404815],
        [8.718611305370995, 49.019748234404815],
        [8.718611305370995, 48.99458739860198],
        [8.679647386702298, 48.99458739860198],
        [8.679647386702298, 49.019748234404815],
    ]
]
poly_dict = json.dumps({"type": "Polygon", "coordinates": Bretten_coords})

In [ ]:
geom = json.loads(poly_dict)
search_geom = shapely.geometry.shape(geom)

m = folium.Map([49.00772676282502, 8.701981844725482], zoom_start=11)
folium.GeoJson(search_geom).add_to(m)
m

In [ ]:
print("Distance search: ", len(db.query_by_geometry(poly_dict, distance=0.0)))

- Do several queries and join results using a logical operator

In [ ]:
# Query example: Select tree species AND
res_fagus = db.query("species", "Acer")
res_ALS = db.query("mode", "TLS")
res_fagus_ALS = db.join([res_fagus, res_ALS], operator="and")
print("Acer with TLS data:\n", res_fagus_ALS)

## Export DB or subset

In [ ]:
db.export_data("tmp/SYSSIFOSS/tmp", trees=[1, 5])  # Export only subset